In [1]:
%load_ext autoreload
# Enable autoreload for all modules
%autoreload 2

In [2]:
from iris_environments.environments import get_environment_builder
import numpy as np
import ipywidgets as widgets
from functools import partial
from pydrake.all import (RigidTransform, Rgba, Sphere, RotationMatrix)

In [3]:
from functools import partial
from pydrake.all import (StartMeshcat,
                         RobotDiagramBuilder,
                         MeshcatVisualizer,
                         LoadModelDirectives,
                         ProcessModelDirectives,
                         RigidTransform,
                         RotationMatrix,
                         MeshcatVisualizerParams,
                         Role,
                         RollPitchYaw,
                         Meldis,
                         AddDefaultVisualization,
                         Box
                         )
import numpy as np
import os

def plant_builder_7dof_bins(usemeshcat = False):
    if usemeshcat:
        #meshcat = StartMeshcat()
        meldis = Meldis()
        meshcat = meldis.meshcat
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    scene_graph = builder.scene_graph()
    parser = builder.parser()
    #parser.package_map().Add("cvisirisexamples", missing directory)
    file_p = "/home/peter/gitcspace/iris_benchmarks/iris_environments"
    directives_file = file_p+"/directives/7dof_bins_example_urdf.yaml"#FindResourceOrThrow() 
    path_repo = file_p#os.path.dirname(os.path.abspath(__file__))
    parser.package_map().Add("iris_environments", path_repo+"/assets")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()
    if usemeshcat:
        #par = MeshcatVisualizerParams()
        #par.role = Role.kIllustration
        #visualizer = MeshcatVisualizer.AddToBuilder(builder.builder(), scene_graph, meshcat, par)
        visualizer = AddDefaultVisualization(builder.builder(), meshcat)
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    diagram.ForcedPublish(diagram_context)
    return plant, scene_graph, diagram, diagram_context, plant_context, meshcat if usemeshcat else None



In [4]:
# import sys
# sys.path.append('../cuciv0/bazel-bin/cuci/src/pybind')
# import pycuci
# cuci_parser = pycuci.URDFParser()
# cuci_parser.register_package("iris_environments", "./iris_environments")
# cuci_parser.parse_directives("iris_environments/directives/7dof_bins_example_urdf.yaml")

In [5]:
from iris_environments.environments import env_names
currname = env_names[6]
plant_builder = get_environment_builder(currname)
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder_7dof_bins(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
#make configuration file 
# import os
# seed_point_file = 'benchmarks/seedpoints/'+currname+'.yml'
# if seed_point_file.split('/')[2] in os.listdir('benchmarks/seedpoints'):
#     file = open(seed_point_file, 'a') 
# else:
#     file = open(seed_point_file, 'w')
#     file.write('seedpoints:\n')

In [ ]:
import xml.etree.ElementTree as ET
from pydrake.all import MultibodyPlant, Sphere, Box, Cylinder, Role, SceneGraph
from pydrake.all import RevoluteJoint, PrismaticJoint, BallRpyJoint, WeldJoint

def drake_to_urdf(plant :MultibodyPlant,
                  scene_graph: SceneGraph,
                  plant_name: str, 
                  urdf_output_path : str):
    

    # Create the root element for the URDF
    robot = ET.Element("robot")
    robot.set("name", plant_name)
    
    # Extract and convert the kinematic tree
    for body_index in range(plant.num_bodies()):
        body = plant.get_body(body_index)
        
        # Skip the world body
        if body.index() == plant.world_body().index():
            continue
        
        # Create a link element for each body
        link = ET.SubElement(robot, "link")
        link.set("name", body.name())
        
        # Add visual and collision geometries
        inspector = scene_graph.model_inspector()
        geometries = plant.GetCollisionGeometriesForBody(body)
        
        for geometry_id in geometries:
            shape = inspector.GetShape(geometry_id)
            
            for role in [Role.kVisual, Role.kProximity]:
                element_name = "visual" if role == Role.kVisual else "collision"
                geometry_element = ET.SubElement(link, element_name)
                
                # Add geometry information
                if isinstance(shape, Sphere):
                    sphere = ET.SubElement(geometry_element, "sphere")
                    sphere.set("radius", str(shape.radius()))
                elif isinstance(shape, Cylinder):
                    cylinder = ET.SubElement(geometry_element, "cylinder")
                    cylinder.set("radius", str(shape.radius()))
                    cylinder.set("length", str(shape.length()))
                elif isinstance(shape, Box):
                    box = ET.SubElement(geometry_element, "box")
                    size = f"{shape.width()} {shape.depth()} {shape.height()}"
                    box.set("size", size)
                # Add more geometry types as needed
        
        # Add joint information
        if body.has_parent_body():
            parent_body = body.parent_body()
            joint = body.get_joint()
            
            joint_element = ET.SubElement(robot, "joint")
            joint_element.set("name", joint.name())
            joint_element.set("type", get_joint_type(joint))
            
            parent = ET.SubElement(joint_element, "parent")
            parent.set("link", parent_body.name())
            
            child = ET.SubElement(joint_element, "child")
            child.set("link", body.name())
            
            # Add joint axis and origin information
            axis = ET.SubElement(joint_element, "axis")
            axis.set("xyz", f"{joint.axis()[0]} {joint.axis()[1]} {joint.axis()[2]}")
            
            origin = ET.SubElement(joint_element, "origin")
            transform = joint.frame_on_parent().GetFixedPoseInBodyFrame()
            translation = transform.translation()
            rotation = transform.rotation().ToQuaternion()
            origin.set("xyz", f"{translation[0]} {translation[1]} {translation[2]}")
            origin.set("rpy", f"{rotation.x()} {rotation.y()} {rotation.z()}")
    
    # Write the URDF to file
    tree = ET.ElementTree(robot)
    tree.write(urdf_output_path, encoding="utf-8", xml_declaration=True)

def get_joint_type(joint):
    if isinstance(joint, RevoluteJoint):
        return "revolute"
    elif isinstance(joint, PrismaticJoint):
        return "prismatic"
    elif isinstance(joint, BallRpyJoint):
        return "ball"
    elif isinstance(joint, WeldJoint):
        return "fixed"
    # Add more joint types as needed
    return "fixed"  # Default to fixed joint


In [7]:
plant.GetJointByName('')

RuntimeError: GetJointByName(): There is no Joint named '' anywhere in the model (valid names in model instance 'iiwa' are: joint1, joint2, joint3, joint4, joint5, joint6, joint7, world_welds_to_base_link; valid names in model instance 'wsg' are: left_finger_sliding_joint, right_finger_sliding_joint, wsg_attach_welds_to_body; valid names in model instance 'shelves' are: shelf_origin_welds_to_shelves_body, top_and_bottom_shelves_body; valid names in model instance 'binR' are: bin_originR_welds_to_bin_base; valid names in model instance 'binL' are: bin_originL_welds_to_bin_base; valid names in model instance 'table' are: table_origin_welds_to_table_body)

# Run this cell and then use the sliders and the button to save the seed points

In [7]:


q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

def write_seed_point_to_file(button):
    col = showres(q)
    if not col:
        line  = '- ['
        for a in q[:-1]:
            line+= str(a)+', '
        line+= str(q[-1])+']\n'
        file.write(line)
        file.flush()
        #write seedpoint to file
    else:
        raise ValueError("That point is in collision")

button = widgets.Button(description="Save")

# Attach the function to the button's click event
button.on_click(write_seed_point_to_file)

# Display the button
display(button)

FloatSlider(value=0.0, description='q0', max=2.9403, min=-2.9403, step=0.001)

FloatSlider(value=0.0, description='q1', max=2.0690999999999997, min=-2.0690999999999997, step=0.001)

FloatSlider(value=0.0, description='q2', max=2.9403, min=-2.9403, step=0.001)

FloatSlider(value=0.0, description='q3', max=2.0690999999999997, min=-2.0690999999999997, step=0.001)

FloatSlider(value=0.0, description='q4', max=2.9403, min=-2.9403, step=0.001)

FloatSlider(value=0.0, description='q5', max=2.0690999999999997, min=-2.0690999999999997, step=0.001)

FloatSlider(value=0.0, description='q6', max=3.0195, min=-3.0195, step=0.001)

Button(description='Save', style=ButtonStyle())

In [6]:
q = np.array([-8.30000e-03,  1.16796e+00,  9.16740e-01,  8.10270e-01,
        8.84370e-01,  1.15137e+00,  6.28890e-01,  6.30620e-01,
       -3.00000e-04,  3.74960e-01,  1.42774e+00,  5.89270e-01,
       -1.43300e-01,  5.32960e-01,  9.91740e-01,  6.33270e-01])

In [7]:
file.close()

# Inspect seed points

In [8]:
import yaml
import time

with open(seed_point_file, 'r') as f:
    seed_points = yaml.safe_load(f)
seed_points = np.array(seed_points['seedpoints'])

for i, s in enumerate(seed_points):
    showres(s)
    print(f" point {i+1} / {len(seed_points)}")
    time.sleep(2)


 point 1 / 10
 point 2 / 10
 point 3 / 10
 point 4 / 10
 point 5 / 10
 point 6 / 10
 point 7 / 10
 point 8 / 10
 point 9 / 10
 point 10 / 10


In [11]:
plant.GetBodyByName("")

RuntimeError: GetRigidBodyByName(): There is no RigidBody named '' anywhere in the model (valid names in model instance 'WorldModelInstance' are: world; valid names in model instance 'allegro_hand' are: hand_root, link_0, link_1, link_10, link_11, link_12, link_13, link_14, link_15, link_2, link_3, link_4, link_5, link_6, link_7, link_8, link_9; valid names in model instance 'block' are: block_body)

In [20]:
showres(seed_points[-3])

False

In [10]:
q

array([ 0.434544, -0.6944  ])